In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Luanda,AO,2021-06-02 19:23:55,-8.8368,13.2343,77.00,83,0,6.91
1,1,Krugersdorp,ZA,2021-06-02 19:26:56,-26.0858,27.7752,47.75,44,53,3.78
2,2,Dingle,PH,2021-06-02 19:26:56,10.9995,122.6711,76.89,91,100,5.26
3,3,Paidha,UG,2021-06-02 19:26:57,2.4167,30.9855,64.29,79,100,5.75
4,4,Butaritari,KI,2021-06-02 19:25:06,3.0707,172.7902,82.94,78,100,15.52


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Luanda,AO,2021-06-02 19:23:55,-8.8368,13.2343,77.00,83,0,6.91
2,2,Dingle,PH,2021-06-02 19:26:56,10.9995,122.6711,76.89,91,100,5.26
4,4,Butaritari,KI,2021-06-02 19:25:06,3.0707,172.7902,82.94,78,100,15.52
6,6,Hilo,US,2021-06-02 19:22:29,19.7297,-155.0900,77.79,78,90,3.44
7,7,Waw,SS,2021-06-02 19:26:58,7.7029,27.9953,79.57,65,100,3.76
9,9,Kuching,MY,2021-06-02 19:26:59,1.5500,110.3333,79.02,89,40,3.44
12,12,Havelock,US,2021-06-02 19:24:27,34.8791,-76.9013,82.98,81,100,5.99
13,13,Santa Rosa,PH,2021-06-02 19:23:52,14.3122,121.1114,81.91,94,100,7.20
17,17,Kapaa,US,2021-06-02 19:23:50,22.0752,-159.3190,81.12,72,1,1.99
18,18,Atuona,PF,2021-06-02 19:23:52,-9.8000,-139.0333,80.02,72,100,15.93


In [10]:
preferred_cities_df.count()

City_ID       196
City          196
Country       196
Date          196
Lat           196
Lng           196
Max Temp      196
Humidity      196
Cloudiness    196
Wind Speed    196
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Luanda,AO,77.00,-8.8368,13.2343,
2,Dingle,PH,76.89,10.9995,122.6711,
4,Butaritari,KI,82.94,3.0707,172.7902,
6,Hilo,US,77.79,19.7297,-155.0900,
7,Waw,SS,79.57,7.7029,27.9953,
9,Kuching,MY,79.02,1.5500,110.3333,
12,Havelock,US,82.98,34.8791,-76.9013,
13,Santa Rosa,PH,81.91,14.3122,121.1114,
17,Kapaa,US,81.12,22.0752,-159.3190,
18,Atuona,PF,80.02,-9.8000,-139.0333,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))